In [1]:
from sys import path
path.append('..')
path.append('../../src/')

import pickle as pkl
from os.path import join, exists

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import imgaug as ia
import imgaug.augmenters as iaa

from mrk_file import MRKFile
from data_structures import Eye
from iso_standard import PhotographicRequirements
from icao_dataset import ICAODataset

from global_config import IMAGE_SIZE, RANDOM_SEED
from local_config import FOLDER_MODELS, FILE_DATASET

%matplotlib inline

In [2]:
PATH_TO_ROOT = '../../'

FOLDER_MODELS = join(PATH_TO_ROOT, FOLDER_MODELS)
FOLDER_IMAGES = join(PATH_TO_ROOT, "data", "celebfaces", "img_align_celeba", "img_align_celeba")
FILE_DATASET = join(PATH_TO_ROOT, "data/celebfaces/dataset_landmark_celebfaces.pkl")

print(FOLDER_MODELS)
print(FOLDER_IMAGES)
print(FILE_DATASET)

../../models/landmark_decoder/
../../data\celebfaces\img_align_celeba\img_align_celeba
../../data/celebfaces/dataset_landmark_celebfaces.pkl


# Data Loading 

In [3]:
df = pd.read_csv(join(PATH_TO_ROOT, "data", "celebfaces", "list_landmarks_align_celeba.csv"))
df = df.sample(n=20000, replace=False, random_state=RANDOM_SEED)
df.reset_index(inplace=True)
df.head()

,index,image_id,lefteye_x,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y
0,93241,093242.jpg,70,108,105,112,98,127,77,153,104,154
1,121413,121414.jpg,69,111,108,112,89,129,74,152,104,152
2,10624,010625.jpg,70,111,108,111,92,141,73,152,103,153
3,82523,082524.jpg,70,112,107,110,74,132,68,153,108,152
4,73192,073193.jpg,70,112,108,111,88,137,67,153,109,152


In [4]:
%%time

IMAGE_SIZE_CELEBA = (218, 178)

new_dataset = ICAODataset()
new_dataset.y_images = []
n_files = df.shape[0]

for i, row in df.iterrows():
    print(f"{i + 1} of {n_files}", end="\r")
    image_path = join(FOLDER_IMAGES, row.image_id)
    
    im = cv2.imread(image_path, cv2.IMREAD_COLOR)
    landmarks = [[row.lefteye_x, row.lefteye_y], [row.righteye_x, row.righteye_y]]
    keypoints = ia.KeypointsOnImage.from_xy_array(landmarks, shape=IMAGE_SIZE_CELEBA)
    
    im_resize = cv2.resize(im, IMAGE_SIZE, interpolation=cv2.INTER_AREA)
    keypoints_resize = keypoints.on(im_resize)
    # ICAO uses the person-in-picture's point-of-view, while CELEBA is the opposite
    eye_left = (keypoints_resize.keypoints[1].x, keypoints_resize.keypoints[1].y)
    eye_right = (keypoints_resize.keypoints[0].x, keypoints_resize.keypoints[0].y)
    
#     cv2.circle(im_resize, eye_left, radius=1, color=(0, 255, 0), thickness=2)
#     cv2.circle(im_resize, eye_right, radius=1, color=(0, 0, 255), thickness=2)

    mrk_file = MRKFile(
        right_eye=Eye(*eye_right, *eye_right),
        left_eye=Eye(*eye_left, *eye_left),
        photo_reqs=PhotographicRequirements(),
    )
    
#     im_landmark = np.zeros((*IMAGE_SIZE, 1))
#     im_landmark[eye_left[1], eye_left[0]] = 255
#     im_landmark[eye_right[1], eye_right[0]] = 255

    new_dataset.image_files.append(image_path)
    new_dataset.images.append(im_resize)
    new_dataset.mrks.append(mrk_file)
#     new_dataset.y_images.append(im_landmark)

Wall time: 33.5 s


In [5]:
FILE_OUTPUT = join(PATH_TO_ROOT, "data", "celebfaces", "dataset.pkl")

pkl.dump(new_dataset, open(FILE_OUTPUT, "wb"), protocol=4)